In [21]:
from denoiser import pretrained
from IPython.display import display, Audio
import torch
import torchaudio
import os
from scipy.signal import convolve
from denoiser.dsp import convert_audio
import pandas as pd
import random
import librosa
import math
import numpy as np

SAMPLE_RATE = 16000
RIR_files = os.listdir('RIR/')[1:]
BG_files = os.listdir('Bg_sound')
SNR = [i for i in range(0,10)]

In [22]:
def get_audio_list(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory)]

def get_org_audfname(aud_dir, fname):
    return os.path.join(aud_dir, fname.split('/')[-1][2:])

def get_context_label(fname):
    return (int)(fname.split('/')[-1][0])

def normalise_npArray(arr):
    arr = (arr - min(arr))/(max(arr) - min(arr))
    return arr

def get_convolved_audio(aud_dir, filename):
    
    label = get_context_label(filename)
    org_audio_fname = get_org_audfname(aud_dir, filename)
    rir_fname = os.path.join('RIR/', RIR_files[label])
    
    org_wav, sr = librosa.load(org_audio_fname, sr=SAMPLE_RATE)
    #org_wav = normalise_npArray(org_wav)
    rir_wav, sr = librosa.load(rir_fname, sr=SAMPLE_RATE)
    #rir_wav = normalise_npArray(rir_wav)

    convolved_audio = convolve(org_wav, rir_wav)[0:org_wav.shape[0]]
    #convolved_audio = normalise_npArray(convolved_audio)
    return convolved_audio

def get_noise_from_sound(signal,noise,SNR):
    
    RMS_s=math.sqrt(np.mean(signal**2))
    #required RMS of noise
    RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/10)))
    
    #current RMS of noise
    RMS_n_current=math.sqrt(np.mean(noise**2))
    if RMS_n_current == 0:
        RMS_n_current = 1
    noise=noise*(RMS_n/RMS_n_current)
    
    return noise

def add_noise(audio, noise):

    SNR_list = [0, 5, 10, 15, 20]
    SNR_choice = random.choice(SNR_list)
    #print(SNR_choice)
    noise = get_noise_from_sound(audio, noise[0:audio.shape[0]], SNR_choice)
    noise = normalise_npArray(noise)
    noisy_audio = audio + noise
    return noisy_audio

def get_noisy_audio(aud_dir, aud_fname):
    label = get_context_label(aud_fname)

    convolved_audio = get_convolved_audio(aud_dir, aud_fname)
    bg_file = os.path.join('Bg_sound/', BG_files[2-label])
    bg_wav, sr = librosa.load(bg_file, sr=SAMPLE_RATE)
    noisy_audio = add_noise(convolved_audio, bg_wav)
    #noisy_audio = normalise_npArray(noisy_audio)

    return noisy_audio

In [61]:
aud_dir2 = '../../LibriVox_Kaggle/serapionsbruederauswahl/'
aud_dir1 = '../../LibriVox_Kaggle/meisterfloh/'
aud_dir0 = '../../LibriVox_Kaggle/achtgesichterambiwasse/'

aud_list2 = get_audio_list(r'../../LibriVox_Kaggle/serapionsbruederauswahl/n1n2/')
aud_list0 = get_audio_list(r'../../LibriVox_Kaggle/achtgesichterambiwasse/n1n2/')
aud_list1 = get_audio_list(r'../../LibriVox_Kaggle/meisterfloh/n1n2/')

aud_list1.sort()
aud_list0.sort()
aud_list2.sort()

aud_list1.pop()
aud_list0.pop()
aud_list2.pop()

'../../LibriVox_Kaggle/serapionsbruederauswahl/n1n2/formatted'

In [37]:
aud_list = []
aud_list0.extend(aud_list1)

In [51]:
aud_list0.extend(aud_list2)
len(aud_list0)

7427

In [25]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

LANG_ID = 'de'
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-german"
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

def make_wav2vec2_preds(audio):
    inputs = processor(audio, sampling_rate=SAMPLE_RATE, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    preds_sentence = processor.batch_decode(predicted_ids)

    #print(preds_sentence)
    return preds_sentence[0]

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [24]:
denoiser_model = pretrained.dns64().cuda()
def denoiser(audio):
    audio = convert_audio(audio.cuda(), SAMPLE_RATE, denoiser_model.sample_rate, denoiser_model.chin)
    with torch.no_grad():
        denoised = denoiser_model(audio[None])[0]
    return denoised.cpu()

In [56]:
samp = random.choice(aud_list0)

wav, sr = torchaudio.load(samp)
wav = denoiser(wav)
display(Audio(wav, rate=sr))

In [77]:
#RIR_files = os.listdir('RIR/')[1:]
wav2vec2_preds = []
'''audio = get_noisy_audio(aud_dir, random.choice(aud_list))

display(Audio(audio, rate=SAMPLE_RATE))
make_wav2vec2_preds(audio)'''

for i in range(0, len(aud_list2)):
    noisy_audio = get_noisy_audio(aud_dir2, aud_list2[i])
    noisy_audio = torch.from_numpy(noisy_audio).unsqueeze(0)
    denoised_audio = denoiser(noisy_audio)
    wav2vec2_preds.append(make_wav2vec2_preds(denoised_audio.squeeze()))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [78]:
#full_flist = []
#full_preds = []
full_flist.extend(aud_list2)
full_preds.extend(wav2vec2_preds)

In [79]:
len(full_flist), len(full_preds)

(7427, 7427)

In [80]:
nDf = pd.DataFrame()
nDf['filename'] = full_flist
nDf['preds'] = full_preds

nDf

,filename,preds
0,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,annabe altab alite beschrug den neasanschlüsse...
1,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,ihr wer abendeuko ist für die frühburg eines n...
2,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,es erringern nur ihr schandtuen dei deren sinn...
3,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,ovry etwanach dritte den bisioms em fichermits...
4,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,asende bei mauntesrals kebärt diern ansenkersä...
...,...,...
7422,../../LibriVox_Kaggle/serapionsbruederauswahl/...,und mitten in diesem draure lag der leichnam e...
7423,../../LibriVox_Kaggle/serapionsbruederauswahl/...,auhenlicherbare untergoen
7424,../../LibriVox_Kaggle/serapionsbruederauswahl/...,bewies er das benkenden alete bis aben und war...
7425,../../LibriVox_Kaggle/serapionsbruederauswahl/...,betröstet- und erhalter fährte er zu rupf nach...


In [81]:
nDf.to_csv('new2.csv', index=False)

In [9]:
df = pd.read_csv('new.csv')
df

,filename,preds
0,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,anlace hatte alle egrabbageunk die ein jasanis...
1,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,ihr aaren war kurz wie die funke berliste noch...
2,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,alsgingen nur ihr schatten und der genersäng u...
3,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,da sie in der nacht hatter den geschlossenen p...
4,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,hasarke war das reichsvor mädchen am siegasen ...
...,...,...
5704,../../LibriVox_Kaggle/meisterfloh/n1n2/2_meist...,doch werde ich euch treu und freundlich gewuch...
5705,../../LibriVox_Kaggle/meisterfloh/n1n2/2_meist...,meister floh hatte während dieser zeit seine n...
5706,../../LibriVox_Kaggle/meisterfloh/n1n2/2_meist...,wirklich soll sich auch meister flo in der fam...
5707,../../LibriVox_Kaggle/meisterfloh/n1n2/2_meist...,vorzüglich tätig gewesen sein als nach jahres ...


In [15]:
df['filename'] = pd.concat([df['filename'],pd.Series(aud_list)], ignore_index=True)
df['preds'] = pd.concat([df['preds'], pd.Series(wav2vec2_preds)], ignore_index=True)

'../../LibriVox_Kaggle/achtgesichterambiwasse/n1n2/0_achtgesichterambiwasse_0000.wav'

In [19]:
new = pd.DataFrame()

new['filename'] = pd.concat([df['filename'],pd.Series(aud_list)], ignore_index=True)
new['preds'] = pd.concat([df['preds'], pd.Series(wav2vec2_preds)], ignore_index=True)

,filename,preds
0,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,anlace hatte alle egrabbageunk die ein jasanis...
1,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,ihr aaren war kurz wie die funke berliste noch...
2,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,alsgingen nur ihr schatten und der genersäng u...
3,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,da sie in der nacht hatter den geschlossenen p...
4,../../LibriVox_Kaggle/achtgesichterambiwasse/n...,hasarke war das reichsvor mädchen am siegasen ...
...,...,...
7422,../../LibriVox_Kaggle/serapionsbruederauswahl/...,und mitten in diesem kreise lag der leichnam e...
7423,../../LibriVox_Kaggle/serapionsbruederauswahl/...,aurelie war runterlehen
7424,../../LibriVox_Kaggle/serapionsbruederauswahl/...,verliess er das zimmerleitete sich an und warf...
7425,../../LibriVox_Kaggle/serapionsbruederauswahl/...,getröstet und erheitert kehrte er zurück nach ...


In [20]:
new.to_csv('new.csv', index=False)

In [57]:
wav2vec2DF = pd.DataFrame()
wav2vec2DF['filename'] = aud_list0
wav2vec2DF['preds'] = wav2vec2_preds

In [60]:
#sample = '../../LibriVox_Kaggle/achtgesichterambiwasse/n1n2/0_achtgesichterambiwasse_1257.wav'
sample = aud_list[3]
print(sample, get_context_label(sample))
wav, sr = librosa.load(get_org_audfname(aud_dir, sample), sr=22050)

rir_wav, sr_rir = librosa.load(os.path.join('RIR/',random.choice(RIR_files)), sr=22050)
con_wav = convolve(wav, rir_wav)[0:wav.shape[0]]

display(Audio(wav, rate=sr))
display(Audio(con_wav, rate=sr))

../../LibriVox_Kaggle/achtgesichterambiwasse/n1n2/2_achtgesichterambiwasse_0015.wav 2


In [22]:
os.listdir('../../LibriVox_Kaggle/achtgesichterambiwasse/n1n2/')

['1_achtgesichterambiwasse_0007.wav',
 '0_achtgesichterambiwasse_0009.wav',
 '0_achtgesichterambiwasse_0013.wav',
 '2_achtgesichterambiwasse_0015.wav',
 '2_achtgesichterambiwasse_0022.wav',
 '1_achtgesichterambiwasse_0023.wav',
 '2_achtgesichterambiwasse_0027.wav',
 '0_achtgesichterambiwasse_0029.wav',
 '2_achtgesichterambiwasse_0030.wav',
 '0_achtgesichterambiwasse_0032.wav',
 '1_achtgesichterambiwasse_0034.wav',
 '2_achtgesichterambiwasse_0038.wav',
 '2_achtgesichterambiwasse_0039.wav',
 '1_achtgesichterambiwasse_0045.wav',
 '2_achtgesichterambiwasse_0047.wav',
 '2_achtgesichterambiwasse_0049.wav',
 '0_achtgesichterambiwasse_0055.wav',
 '1_achtgesichterambiwasse_0057.wav',
 '2_achtgesichterambiwasse_0058.wav',
 '0_achtgesichterambiwasse_0061.wav',
 '2_achtgesichterambiwasse_0066.wav',
 '1_achtgesichterambiwasse_0070.wav',
 '2_achtgesichterambiwasse_0072.wav',
 '2_achtgesichterambiwasse_0075.wav',
 '2_achtgesichterambiwasse_0078.wav',
 '2_achtgesichterambiwasse_0082.wav',
 '0_achtgesi